In [14]:
import os
import datetime as dt

import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython
from IPython.display import Audio
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split

import tensorflow as tf
print(f'tensorflow version: {tf.__version__}')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

tensorflow version: 2.7.0


In [2]:
tqdm_notebook.pandas()

### Reviewing Dataset

In [ ]:
for index, filename in enumerate(os.listdir('cat')):
    os.rename(f'cat/{filename}', f'cat/{str(index)}.wav')
    
for index,filename in enumerate(os.listdir('dog')):
    os.rename(f'dog/{filename}', f'dog/{str(index)}.wav')

In [ ]:
IPython.display.display(IPython.display.Audio('cat/0.wav'))
IPython.display.display(IPython.display.Audio('cat/1.wav'))

In [ ]:
IPython.display.display(IPython.display.Audio('dog/0.wav'))
IPython.display.display(IPython.display.Audio('dog/1.wav'))

In [ ]:
audio, sr = librosa.load('cat/1.wav')
plt.figure(figsize=(12,5))
librosa.display.waveplot(audio, sr)

In [ ]:
audio, sr = librosa.load('dog/1.wav')
plt.figure(figsize=(12,5))
librosa.display.waveplot(audio, sr)

## Data Preprocessing

In [3]:
cat_audio_list = []
for i in range(len(os.listdir('cat'))):
    cat_audio_list.append(f'cat/{i}.wav')
    
dog_audio_list = []
for i in range(len(os.listdir('dog'))):
    dog_audio_list.append(f'dog/{i}.wav')

In [4]:
df = pd.DataFrame()

In [5]:
cat_audio_data = []
for filename in cat_audio_list:
    cat_audio_data.append({
        'filename': filename,
        'label': 'cat'
    })
df = df.append(pd.DataFrame(cat_audio_data), ignore_index=True)
    
dog_audio_data = []
for filename in dog_audio_list:
    dog_audio_data.append({
        'filename': filename,
        'label': 'dog'
    })
df = df.append(pd.DataFrame(dog_audio_data),  ignore_index=True)

In [6]:
df

,filename,label
0,cat/0.wav,cat
1,cat/1.wav,cat
2,cat/2.wav,cat
3,cat/3.wav,cat
4,cat/4.wav,cat
...,...,...
272,dog/108.wav,dog
273,dog/109.wav,dog
274,dog/110.wav,dog
275,dog/111.wav,dog


In [7]:
df['is_dog'] = pd.get_dummies(df['label'], drop_first=True)

In [8]:
df['audio'] = df['filename'].progress_apply(lambda x: librosa.load(x)[0])
df['sr'] = 22050

  0%|          | 0/277 [00:00<?, ?it/s]

In [9]:
df['mfcc'] = df['audio'].progress_apply(lambda x: librosa.feature.mfcc(x, n_mfcc=12))

  0%|          | 0/277 [00:00<?, ?it/s]

In [10]:
df['mfcc_mean'] = df['mfcc'].progress_apply(lambda x: np.mean(x.T, axis=0))

  0%|          | 0/277 [00:00<?, ?it/s]

In [ ]:
df.to_pickle('metadata.pkl')

## Training Model

In [11]:
df = pd.read_pickle('metadata.pkl')

In [12]:
X = np.vstack(df['mfcc_mean'].to_numpy())
y = df['is_dog'].to_numpy().reshape(-1,1).astype('float32')

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=14)

In [15]:
model = Sequential(
    [
        Input(shape=(12,)),
        Dense(100, activation='relu', name='layer_1'),
        Dropout(0.5),
        Dense(200, activation='relu', name='layer_2'),
        Dropout(0.5),
        Dense(100, activation='relu', name='layer_3'),
        Dropout(0.5),
        Dense(1, activation='sigmoid', name='output_layer'),
    ]
)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_1 (Dense)             (None, 100)               1300      
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 layer_2 (Dense)             (None, 200)               20200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 layer_3 (Dense)             (None, 100)               20100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 output_layer (Dense)        (None, 1)                 1

In [17]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

In [18]:
log_dir = "logs/fit/" + dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
%%time
num_epochs = 300
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='models/best.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(X_train, 
          y_train, 
          batch_size=num_batch_size, 
          epochs=num_epochs, 
          validation_data=(X_test, y_test), 
          callbacks=[checkpointer, tensorboard_callback], 
          verbose=1)

Epoch 1/300
1/7 [===>..........................] - ETA: 2s - loss: 21.0354 - accuracy: 0.4375
Epoch 00001: val_loss improved from inf to 3.30402, saving model to models\best.hdf5
7/7 [==============================] - 1s 33ms/step - loss: 18.4602 - accuracy: 0.4974 - val_loss: 3.3040 - val_accuracy: 0.6548
Epoch 2/300
1/7 [===>..........................] - ETA: 0s - loss: 17.7463 - accuracy: 0.6250
Epoch 00002: val_loss improved from 3.30402 to 0.85865, saving model to models\best.hdf5
7/7 [==============================] - 0s 15ms/step - loss: 16.6027 - accuracy: 0.5389 - val_loss: 0.8587 - val_accuracy: 0.8571
Epoch 3/300
1/7 [===>..........................] - ETA: 0s - loss: 13.8532 - accuracy: 0.5625
Epoch 00003: val_loss did not improve from 0.85865
7/7 [==============================] - 0s 12ms/step - loss: 12.3885 - accuracy: 0.5751 - val_loss: 0.9345 - val_accuracy: 0.8214
Epoch 4/300
1/7 [===>..........................] - ETA: 0s - loss: 13.9097 - accuracy: 0.5000
Epoch 00004:

Epoch 00029: val_loss improved from 0.40961 to 0.38823, saving model to models\best.hdf5
7/7 [==============================] - 0s 14ms/step - loss: 2.0890 - accuracy: 0.7150 - val_loss: 0.3882 - val_accuracy: 0.9167
Epoch 30/300
1/7 [===>..........................] - ETA: 0s - loss: 1.7263 - accuracy: 0.6562
Epoch 00030: val_loss improved from 0.38823 to 0.38245, saving model to models\best.hdf5
7/7 [==============================] - 0s 14ms/step - loss: 1.7431 - accuracy: 0.7358 - val_loss: 0.3825 - val_accuracy: 0.9167
Epoch 31/300
1/7 [===>..........................] - ETA: 0s - loss: 1.0308 - accuracy: 0.7188
Epoch 00031: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 9ms/step - loss: 1.5108 - accuracy: 0.7047 - val_loss: 0.3934 - val_accuracy: 0.9167
Epoch 32/300
1/7 [===>..........................] - ETA: 0s - loss: 2.4912 - accuracy: 0.6250
Epoch 00032: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 10ms/step - l

1/7 [===>..........................] - ETA: 0s - loss: 0.9140 - accuracy: 0.8438
Epoch 00059: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 9ms/step - loss: 0.9201 - accuracy: 0.7772 - val_loss: 0.5569 - val_accuracy: 0.8929
Epoch 60/300
1/7 [===>..........................] - ETA: 0s - loss: 0.6918 - accuracy: 0.7812
Epoch 00060: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 9ms/step - loss: 0.8618 - accuracy: 0.7824 - val_loss: 0.5549 - val_accuracy: 0.9048
Epoch 61/300
1/7 [===>..........................] - ETA: 0s - loss: 0.8804 - accuracy: 0.7188
Epoch 00061: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 9ms/step - loss: 0.9444 - accuracy: 0.7254 - val_loss: 0.5556 - val_accuracy: 0.9048
Epoch 62/300
1/7 [===>..........................] - ETA: 0s - loss: 0.2613 - accuracy: 0.8750
Epoch 00062: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 9ms/step -

1/7 [===>..........................] - ETA: 0s - loss: 0.6968 - accuracy: 0.7812
Epoch 00089: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 9ms/step - loss: 0.7295 - accuracy: 0.7409 - val_loss: 0.5052 - val_accuracy: 0.9405
Epoch 90/300
1/7 [===>..........................] - ETA: 0s - loss: 0.5794 - accuracy: 0.7500
Epoch 00090: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 9ms/step - loss: 0.7168 - accuracy: 0.7461 - val_loss: 0.5034 - val_accuracy: 0.9286
Epoch 91/300
1/7 [===>..........................] - ETA: 0s - loss: 0.6179 - accuracy: 0.7500
Epoch 00091: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 9ms/step - loss: 0.6927 - accuracy: 0.7306 - val_loss: 0.5011 - val_accuracy: 0.9405
Epoch 92/300
1/7 [===>..........................] - ETA: 0s - loss: 0.5988 - accuracy: 0.7812
Epoch 00092: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 9ms/step -

1/7 [===>..........................] - ETA: 0s - loss: 0.6098 - accuracy: 0.8438
Epoch 00119: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 11ms/step - loss: 0.4702 - accuracy: 0.8238 - val_loss: 0.4661 - val_accuracy: 0.9167
Epoch 120/300
1/7 [===>..........................] - ETA: 0s - loss: 0.3492 - accuracy: 0.8750
Epoch 00120: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 10ms/step - loss: 0.5128 - accuracy: 0.7772 - val_loss: 0.4640 - val_accuracy: 0.9167
Epoch 121/300
1/7 [===>..........................] - ETA: 0s - loss: 0.4726 - accuracy: 0.7188
Epoch 00121: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 9ms/step - loss: 0.5057 - accuracy: 0.7824 - val_loss: 0.4545 - val_accuracy: 0.9167
Epoch 122/300
1/7 [===>..........................] - ETA: 0s - loss: 0.3883 - accuracy: 0.7500
Epoch 00122: val_loss did not improve from 0.38245
7/7 [==============================] - 0s 9ms/s

...

7/7 [==============================] - 0s 13ms/step - loss: 0.4074 - accuracy: 0.8497 - val_loss: 0.3380 - val_accuracy: 0.9167
Epoch 178/300
1/7 [===>..........................] - ETA: 0s - loss: 0.2888 - accuracy: 0.9062
Epoch 00178: val_loss improved from 0.33803 to 0.33454, saving model to models\best.hdf5
7/7 [==============================] - 0s 13ms/step - loss: 0.3637 - accuracy: 0.8446 - val_loss: 0.3345 - val_accuracy: 0.9167
Epoch 179/300
1/7 [===>..........................] - ETA: 0s - loss: 0.3409 - accuracy: 0.8750
Epoch 00179: val_loss improved from 0.33454 to 0.33020, saving model to models\best.hdf5
7/7 [==============================] - 0s 14ms/step - loss: 0.3917 - accuracy: 0.8290 - val_loss: 0.3302 - val_accuracy: 0.9167
Epoch 180/300
1/7 [===>..........................] - ETA: 0s - loss: 0.4121 - accuracy: 0.8125
Epoch 00180: val_loss improved from 0.33020 to 0.32820, saving model to models\best.hdf5
7/7 [==============================] - 0s 13ms/step - loss: 0.36

Epoch 207/300
1/7 [===>..........................] - ETA: 0s - loss: 0.7278 - accuracy: 0.8438
Epoch 00207: val_loss did not improve from 0.32035
7/7 [==============================] - 0s 9ms/step - loss: 0.4098 - accuracy: 0.8653 - val_loss: 0.3472 - val_accuracy: 0.9167
Epoch 208/300
1/7 [===>..........................] - ETA: 0s - loss: 0.5019 - accuracy: 0.7500
Epoch 00208: val_loss did not improve from 0.32035
7/7 [==============================] - 0s 9ms/step - loss: 0.3679 - accuracy: 0.8601 - val_loss: 0.3463 - val_accuracy: 0.9048
Epoch 209/300
1/7 [===>..........................] - ETA: 0s - loss: 0.1857 - accuracy: 0.9688
Epoch 00209: val_loss did not improve from 0.32035
7/7 [==============================] - 0s 9ms/step - loss: 0.3282 - accuracy: 0.8497 - val_loss: 0.3430 - val_accuracy: 0.9048
Epoch 210/300
1/7 [===>..........................] - ETA: 0s - loss: 0.3705 - accuracy: 0.8438
Epoch 00210: val_loss did not improve from 0.32035
7/7 [==============================

Epoch 237/300
1/7 [===>..........................] - ETA: 0s - loss: 0.4371 - accuracy: 0.9688
Epoch 00237: val_loss did not improve from 0.32035
7/7 [==============================] - 0s 9ms/step - loss: 0.4620 - accuracy: 0.8808 - val_loss: 0.3356 - val_accuracy: 0.9167
Epoch 238/300
1/7 [===>..........................] - ETA: 0s - loss: 0.4155 - accuracy: 0.8438
Epoch 00238: val_loss did not improve from 0.32035
7/7 [==============================] - 0s 9ms/step - loss: 0.3700 - accuracy: 0.8653 - val_loss: 0.3452 - val_accuracy: 0.9286
Epoch 239/300
1/7 [===>..........................] - ETA: 0s - loss: 0.2561 - accuracy: 0.8750
Epoch 00239: val_loss did not improve from 0.32035
7/7 [==============================] - 0s 9ms/step - loss: 0.4069 - accuracy: 0.8446 - val_loss: 0.3939 - val_accuracy: 0.9286
Epoch 240/300
1/7 [===>..........................] - ETA: 0s - loss: 0.2682 - accuracy: 0.8125
Epoch 00240: val_loss did not improve from 0.32035
7/7 [==============================

Epoch 267/300
1/7 [===>..........................] - ETA: 0s - loss: 0.1561 - accuracy: 0.9688
Epoch 00267: val_loss did not improve from 0.32035
7/7 [==============================] - 0s 9ms/step - loss: 0.3013 - accuracy: 0.8860 - val_loss: 0.3223 - val_accuracy: 0.9167
Epoch 268/300
1/7 [===>..........................] - ETA: 0s - loss: 0.4880 - accuracy: 0.7188
Epoch 00268: val_loss improved from 0.32035 to 0.31661, saving model to models\best.hdf5
7/7 [==============================] - 0s 13ms/step - loss: 0.3427 - accuracy: 0.8497 - val_loss: 0.3166 - val_accuracy: 0.9167
Epoch 269/300
1/7 [===>..........................] - ETA: 0s - loss: 0.2283 - accuracy: 0.9062
Epoch 00269: val_loss improved from 0.31661 to 0.31249, saving model to models\best.hdf5
7/7 [==============================] - 0s 15ms/step - loss: 0.2916 - accuracy: 0.8549 - val_loss: 0.3125 - val_accuracy: 0.9167
Epoch 270/300
1/7 [===>..........................] - ETA: 0s - loss: 0.3695 - accuracy: 0.8750
Epoch 00

1/7 [===>..........................] - ETA: 0s - loss: 0.2351 - accuracy: 0.9375
Epoch 00296: val_loss did not improve from 0.31011
7/7 [==============================] - 0s 10ms/step - loss: 0.3288 - accuracy: 0.8653 - val_loss: 0.3165 - val_accuracy: 0.9048
Epoch 297/300
1/7 [===>..........................] - ETA: 0s - loss: 0.3545 - accuracy: 0.8125
Epoch 00297: val_loss did not improve from 0.31011
7/7 [==============================] - 0s 10ms/step - loss: 0.2534 - accuracy: 0.8912 - val_loss: 0.3120 - val_accuracy: 0.9048
Epoch 298/300
1/7 [===>..........................] - ETA: 0s - loss: 0.1605 - accuracy: 0.9375
Epoch 00298: val_loss improved from 0.31011 to 0.30684, saving model to models\best.hdf5
7/7 [==============================] - 0s 15ms/step - loss: 0.3444 - accuracy: 0.8653 - val_loss: 0.3068 - val_accuracy: 0.9048
Epoch 299/300
1/7 [===>..........................] - ETA: 0s - loss: 0.1660 - accuracy: 0.9375
Epoch 00299: val_loss improved from 0.30684 to 0.30298, sav

In [20]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [21]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 9296), started 0:15:51 ago. (Use '!kill 9296' to kill it.)